In [41]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

GLOBAL_PATH = '../'
DATA_PATH = '../Data/'
OUTPUT_PATH = '../Output/'

In [42]:
### Read in the datasets ###

df_cps20 = pd.read_csv(DATA_PATH + 'estimatesCPS2020.csv')
df_cps21 = pd.read_csv(DATA_PATH + 'estimatesCPS2021.csv')
df_medicaid = pd.read_csv(DATA_PATH + 'medicaid19.csv')
df_fiscal = pd.read_csv(DATA_PATH + 'covid_aid.csv')
df_gdp = pd.read_csv(DATA_PATH + 'gdp.csv')
df_minwage = pd.read_csv(DATA_PATH + 'minwage.csv')

In [43]:
### Append the 2020 and 2021 CPS datasets ###

# Append the CPS datasets, use concat
df_cps20['Year'] = 2020
df_cps21['Year'] = 2021
df_cps = pd.concat([df_cps20, df_cps21], axis=0)
df_cps.shape

(1224, 14)

In [44]:
### Merge the CPS and with other datasets based on State and Dates ###

# 1. Merge the medicaid dataset with the fiscal aid dataset
df_medicaid_fiscal = pd.merge(df_medicaid, df_fiscal, on='State')
df_medicaid_fiscal.shape # (51, 9)
df_medicaid_fiscal.head()

,State,Medicaid_Year,Medicaid_Services,CHIP,Medicaid_Admin,Medicaid_Total,Award_Obligations,Award_Outlays,Num_Awards
0,Alabama,2019,5.880234e+09,3.913668e+08,2.162238e+08,6.487824e+09,2.710000e+10,2.460000e+10,297743
1,Alaska,2019,2.096340e+09,3.696655e+07,1.473289e+08,2.280636e+09,1.220000e+10,1.090000e+10,47985
2,Arizona,2019,1.316787e+10,2.343925e+08,3.142175e+08,1.371648e+10,4.210000e+10,4.010000e+10,335840
3,Arkansas,2019,6.842931e+09,1.894763e+08,4.006915e+08,7.433099e+09,1.430000e+10,1.350000e+10,165436
4,California,2019,8.785598e+10,3.650584e+09,6.243263e+09,9.774983e+10,2.764000e+11,2.523000e+11,2732743


In [45]:
# 2. Merge the minwage dataset with the gdp dataset
df_minwage_gdp = pd.merge(df_minwage, df_gdp, on='State')
df_minwage_gdp.shape # (51, 33)
# keep columns State, MinWage20, MinWage21, GDP20, GDP21
df_minwage_gdp = df_minwage_gdp[['State', 'MinWage20', 'MinWage21', 'GDP20', 'GDP21']]
df_minwage_gdp.head()

,State,MinWage20,MinWage21,GDP20,GDP21
0,Alabama,7.25,7.25,1.998808e+11,2.099793e+11
1,Alaska,10.19,10.34,5.070520e+10,5.086940e+10
2,Arizona,12.00,12.15,3.271780e+11,3.476560e+11
3,Arkansas,10.00,11.00,1.172682e+11,1.233473e+11
4,California,13.00,14.00,2.667221e+12,2.874731e+12


In [46]:
# 3. Merge df_cps and df_medicaid_fiscal on state

# but the state in df_cps is in number format
# so we need to convert the state in df_cps to string
State_dict = {1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware',
            11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 
            20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota',
            28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey',
            35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon',
            42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas', 49: 'Utah',
            50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'}

df_cps['State'] = df_cps['State'].map(State_dict)

df_cps_medicaid_fiscal = pd.merge(df_cps, df_medicaid_fiscal, on='State')
df_cps_medicaid_fiscal.shape # (1224, 22) 1224 = 51 * 12 * 2


(1224, 22)

In [47]:
df_cps_medicaid_fiscal.head()
# remove Unnamed: 0 column
df_cps_medicaid_fiscal = df_cps_medicaid_fiscal.drop(columns=['Unnamed: 0'])

In [50]:
# 4. Merge df_cps_medicaid_fiscal and df_minwage_gdp on state
df_final = pd.merge(df_cps_medicaid_fiscal, df_minwage_gdp, on='State')
df_final.shape # (1224, 26)

# if df_cps_medicaid_fiscal Year is 2020, then use MinWage20 and GDP20 from df_minwage_gdp
# if df_cps_medicaid_fiscal Year is 2021, then use MinWage21 and GDP21 from df_minwage_gdp
df_final['MinWage'] = np.where(df_final['Year'] == 2020, df_final['MinWage20'], df_final['MinWage21'])
df_final['GDP'] = np.where(df_final['Year'] == 2020, df_final['GDP20'], df_final['GDP21'])

# drop MinWage20, MinWage21, GDP20, GDP21
df_final = df_final.drop(columns=['MinWage20', 'MinWage21', 'GDP20', 'GDP21'])

# check columns
df_final.columns




Index(['Month', 'State', 'Civil_Pop', 'LaborForce_Pop', 'Unemp_Pop', 'Emp_Pop',
       'Emp_Rate', 'Unemp_Rate', 'LF_Particip_Rate', 'Emp_Pop_Ratio',
       'Union_Pop', 'Union_Rate', 'Year', 'Medicaid_Year', 'Medicaid_Services',
       'CHIP', 'Medicaid_Admin', 'Medicaid_Total', 'Award_Obligations',
       'Award_Outlays', 'Num_Awards', 'MinWage', 'GDP'],
      dtype='object')

In [51]:
df_final.head()

,Month,State,Civil_Pop,LaborForce_Pop,Unemp_Pop,Emp_Pop,Emp_Rate,Unemp_Rate,LF_Particip_Rate,Emp_Pop_Ratio,Union_Pop,Union_Rate,Year,Medicaid_Year,Medicaid_Services,CHIP,Medicaid_Admin,Medicaid_Total,Award_Obligations,Award_Outlays,Num_Awards,MinWage,GDP
0,1,Alabama,3.871113e+06,2.191401e+06,71895.1913,2.119506e+06,96.719214,3.280786,56.609074,54.751851,87503.6079,3.993044,2020,2019,5.880234e+09,391366847.0,216223796.0,6.487824e+09,2.710000e+10,2.460000e+10,297743,7.25,1.998808e+11
1,2,Alabama,3.872006e+06,2.204607e+06,54473.3402,2.150134e+06,97.529114,2.470886,56.937082,55.530231,154302.9432,6.999113,2020,2019,5.880234e+09,391366847.0,216223796.0,6.487824e+09,2.710000e+10,2.460000e+10,297743,7.25,1.998808e+11
2,3,Alabama,3.873022e+06,2.201768e+06,71239.6282,2.130529e+06,96.764436,3.235564,56.848848,55.009467,205893.2337,9.351267,2020,2019,5.880234e+09,391366847.0,216223796.0,6.487824e+09,2.710000e+10,2.460000e+10,297743,7.25,1.998808e+11
3,4,Alabama,3.874113e+06,2.169832e+06,295493.1542,1.874339e+06,86.381749,13.618251,56.008484,48.381108,170584.1993,7.861632,2020,2019,5.880234e+09,391366847.0,216223796.0,6.487824e+09,2.710000e+10,2.460000e+10,297743,7.25,1.998808e+11
4,5,Alabama,3.875399e+06,2.241937e+06,203557.9046,2.038379e+06,90.920445,9.079555,57.850480,52.597914,173450.7058,7.736645,2020,2019,5.880234e+09,391366847.0,216223796.0,6.487824e+09,2.710000e+10,2.460000e+10,297743,7.25,1.998808e+11


In [52]:
# save the final data
df_final.to_csv(DATA_PATH + 'final_data.csv', index=False)